In [ ]:
#just to test old eng/fra version
MAX_LENGTH = 10

eng_prefixes = (
    "i am ", "i m ",
    "he is", "he s ",
    "she is", "she s",
    "you are", "you re ",
    "we are", "we re ",
    "they are", "they re "
)


def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and \
        len(p[1].split(' ')) < MAX_LENGTH and \
        p[1].startswith(eng_prefixes)


def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

In [ ]:
import translator
import torch
%matplotlib inline


#need to run these both to avoid passing device around everywhere
translator.init_device()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    

input_lang, output_lang, pairs = translator.prepareData('eng', 'fra', True)

pairs = filterPairs(pairs)

hidden_size = 256
encoder1 = translator.EncoderRNN(input_lang.n_words, hidden_size).to(device)
#attn_decoder1 =translator.DecoderRNN(hidden_size, output_lang.n_words).to(device)
attn_decoder1 = translator.BahdanauAttnDecoderRNN(hidden_size, output_lang.n_words, n_layers=1, dropout_p=0.1).to(device)

iters = 50000# 75000
print_int = 5000
plot_int = 1000
lr = 0.01
max_length = 10
teacher_forcing_ratio = 0.5

#tokenize and convert to tensors
token_pairs = [translator.tensorsFromPair(pair, input_lang, output_lang) for pair in pairs]
    
plot_losses = translator.trainIters(token_pairs, encoder1, attn_decoder1, iters, max_length, teacher_forcing_ratio, lr, print_every=print_int, plot_every=plot_int)

translator.showPlot(plot_losses)

translator.evaluateRandomly(pairs, max_length, input_lang, output_lang, encoder1, attn_decoder1)
